## Work Notebook
### Algorithm of Hybrid Rec Engine - find similar products according a tag classifier followed by cosine similarity by Spacy embeddings of reviewText 

In [208]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm

In [167]:
#loading downloaded data
df=pd.read_json('~/Downloads/reviews_Electronics_5.json', lines=True)

  0%|                          | 200/1689188 [12:23:59<104716:03:48, 223.20s/it]


### Loading pickled webscraping files

In [170]:
with open('load_df.pickle', 'rb') as file:
    load_df = pickle.load(file)

### We will train a model that classifies user input (item) into appropriate tag labels 

In [171]:
load_df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,item,tag,price
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5,Gotta have GPS!,1370131200,"06 2, 2013",[Rand McNally 528881469 7-inch Intelliroute TN...,Trucking GPS,$399.99
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1,Very Disappointed,1290643200,"11 25, 2010",[Rand McNally 528881469 7-inch Intelliroute TN...,Trucking GPS,$399.99
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3,1st impression,1283990400,"09 9, 2010",[Rand McNally 528881469 7-inch Intelliroute TN...,Trucking GPS,$399.99
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2,"Great grafics, POOR GPS",1290556800,"11 24, 2010",[Rand McNally 528881469 7-inch Intelliroute TN...,Trucking GPS,$399.99
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1,"Major issues, only excuses for support",1317254400,"09 29, 2011",[Rand McNally 528881469 7-inch Intelliroute TN...,Trucking GPS,$399.99


In [295]:
df_mod=load_df[['item','tag']]


In [296]:
item_list=[]

for i in range(702972):
    try:
        if len(df_mod.item[i][0]) >= 1:
            item_list.append(df_mod.item[i][0])
    except:
        item_list.append(np.nan)
        continue
        


In [297]:
df_mod['item']=pd.Series(item_list)

/var/folders/48/3rfqt3q17zj4zsvn_9rh50br0000gn/T/ipykernel_10716/1714449863.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mod['item']=pd.Series(item_list)


In [305]:
len(df_mod.tag.unique())

1032

In [318]:
df_mod.dropna(inplace=True)

/Users/jeffreyng/miniforge3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [342]:
df_mod.shape

(693083, 2)

In [343]:
y=df_mod.tag

In [344]:
X=df_mod.item

In [339]:
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import re
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

In [345]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

In [331]:
# created a NLP processing function that includes all the steps of corpus cleaning and creation and
# vectorization

def NLP_process(data):
    BoW=[]
    for word in data:
        BoW.append(word)
    word_bank= ' '.join(BoW)
    word_bank= word_bank.lower()
    from nltk.tokenize import RegexpTokenizer
    tokenizer = RegexpTokenizer(r'[a-zA-Z]+')
    tokenized_tweets= tokenizer.tokenize(word_bank)

    sw_list = stopwords.words('english')
    sw_list += list(string.punctuation)
    sw_list += ["''", '""', '...', '``', '’', '“', '’', '”', '‘', '‘', '©', '#', '@', 
            'said', 'one', 'com', '-', '–', '—', '.']
    sw_list += ['www', 'the', 'to', 'and', 'a', 's', 't','is', 'you', 'of', 'i', 
            'in', 'this', 'are', 'it', 'for', 'https', 'com', 
             'p',]
    sw_set = set(sw_list)
    filtered_tweets=[]
    for word in tokenized_tweets:
        if word not in sw_set:
            filtered_tweets.append(word)
#     
    lemmatizer=WordNetLemmatizer()
    X_lemmatized_tweets=[]
    for w in filtered_tweets:
        X_lemmatized_tweets.append(lemmatizer.lemmatize(w))
    X_lemmatized_tweets
    
    tfidf = TfidfVectorizer(stop_words=sw_list, ngram_range=(1,1), lowercase=True)
    tfidf_data_test_lem = tfidf.fit_transform(X_lemmatized_tweets)
    return tfidf_data_test_lem 

In [334]:
X_b=NLP_process(X_train)

In [335]:

def NLP_process_t(data):
    BoW=[]
    for word in data:
        BoW.append(word)
    word_bank= ' '.join(BoW)
    word_bank= word_bank.lower()
    from nltk.tokenize import RegexpTokenizer
    tokenizer = RegexpTokenizer(r'[a-zA-Z]+')
    tokenized_tweets= tokenizer.tokenize(word_bank)

    sw_list = stopwords.words('english')
    sw_list += list(string.punctuation)
    sw_list += ["''", '""', '...', '``', '’', '“', '’', '”', '‘', '‘', '©', '#', '@', 
            'said', 'one', 'com', '-', '–', '—', '.']
    sw_list += ['www', 'the', 'to', 'and', 'a', 's', 't','is', 'you', 'of', 'i', 
            'in', 'this', 'are', 'it', 'for', 'https', 'com', 
             'p',]
    sw_set = set(sw_list)
    filtered_tweets=[]
    for word in tokenized_tweets:
        if word not in sw_set:
            filtered_tweets.append(word)
#     
    lemmatizer=WordNetLemmatizer()
    X_lemmatized_tweets=[]
    for w in filtered_tweets:
        X_lemmatized_tweets.append(lemmatizer.lemmatize(w))
    X_lemmatized_tweets
    
    tfidf = TfidfVectorizer(stop_words=sw_list, ngram_range=(1,1), lowercase=True)
    tfidf_data_test_lem = tfidf.transform(X_lemmatized_tweets)
    return tfidf_data_test_lem 

In [336]:
X_t=NLP_process_t(X_test)

NotFittedError: The TF-IDF vectorizer is not fitted

In [348]:
#using tf_idf vectorizer
stop_words = set(stopwords.words("english"))
tfidf = TfidfVectorizer(stop_words=stop_words, ngram_range=(1,1), lowercase=True)
tfidf_data_train_lem = tfidf.fit_transform(X_train)
tfidf_data_test_lem = tfidf.transform(X_test)

In [356]:
from sklearn.naive_bayes import MultinomialNB  
classifier = MultinomialNB(alpha=0)  
classifier.fit(tfidf_data_train_lem, y_train)  

/Users/jeffreyng/miniforge3/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(


MultinomialNB(alpha=0)

In [357]:
y_preds=classifier.predict(tfidf_data_test_lem)

In [353]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn import metrics

In [358]:
print(metrics.accuracy_score(y_test, y_preds))
print(metrics.f1_score(y_test, y_preds, average='weighted'))


0.9805002272448545
0.9803130732680493


In [ ]:
classifier

### We will find the sentiment score fro each review 

In [37]:
df.shape

(63001, 9)

In [53]:
df.reviewText

0          0    We got this GPS for my husband who is an ...
5          5    I am using this with a Nook HD+. It works...
10         10    This item is just as was described in th...
18         18     This is a great buy, compared to a $60 ...
237        237    The Nook Tablet, in both the 16gb versi...
                                 ...                        
1689106    1689106    The Sabrent CR-CCU3 Card reader is ...
1689125    1689125    Great hub for carrying around if yo...
1689142    1689142    This is an interesting device in th...
1689162    1689162    ...all part of the package that com...
1689183    1689183    Burned these in before listening to...
Name: reviewText, Length: 63001, dtype: object

In [71]:
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [95]:
sent=[sid.polarity_scores(x) for x in tqdm(df.reviewText)]

100%|████████████████████████████████| 1689188/1689188 [31:06<00:00, 904.93it/s]


In [110]:
df['vader_score']=[sent[x].get('compound') for x in range(len(sent))]

In [113]:
# Mapped Vader scores to integer counts for easier manipulation 
col         = 'vader_score'
conditions  = [ df[col] >= .15, (df[col] < .1) & (df[col]> -.1), df[col] <= -.1 ]
choices     = [ 1, 0, -1 ]
df["vader_score"] = np.select(conditions, choices, default=np.nan)  

In [115]:
#final mapping process
df['vader_score']= df['vader_score'].map({-1:'negative', 0: 'neutral', 1:'positive'})

In [117]:
df.vader_score.value_counts()

positive    1399723
negative     194525
neutral       80603
Name: vader_score, dtype: int64

### We will use Spacy Embeddings for the df.reviewText

In [118]:
#importing spacy 
import spacy
nlp = spacy.load("en_core_web_lg")

In [119]:
#tokenizer
spacy_tokenizer = nlp.tokenizer

In [120]:
#function to take reviewText column an return word embedding
def prep(x):
    z=spacy_tokenizer(x) 
    z=nlp(z).vector.reshape(300,)
    return z


In [165]:
words=tdqm([prep(x) for x in df.reviewText])

NameError: name 'tdqm' is not defined

In [125]:
import gensim
# set the correct path to the file on your machine
model = gensim.models.KeyedVectors.load_word2vec_format('~/Downloads/crawl-300d-2M.vec', binary=False)

In [145]:
def get_mean_vector(word2vec_model, words):
    # remove out-of-vocabulary words
    words = [word for word in words if word in word2vec_model.key_to_index]
    if len(words) >= 1:
        return np.mean(word2vec_model[words], axis=0)
    else:
        return []

In [156]:
vec = [get_mean_vector(model, df.reviewText[i]) for i in tqdm(range(len(df.reviewText)))]
vec

In [ ]:
len(vec)

In [158]:
len(vec[0])

300

In [ ]:
def tagger(x):
    y_pred=model.predict(x)
    new_df=df[df.tag==str(y_pred)]
    new_df

In [ ]:
def cs_calc(x):

In [ ]:
model=Models.Sequential()
model1 = Sequential()

model2 = Model(inputs=[model.input, model1.input], outputs=x)